
# Application analytics project #1 
```
Current project is about analysis of Apple Store and Google Play application dataset to gain usefull insights.
Main goal - to identify the most relevant type of application to attract new users.
```

In [1]:
from csv import reader 

open_file=open('AppleStore.csv')
read_file=reader(open_file)
ios=list(read_file)
ios_header=ios[0]
ios=ios[1:]

google_store=open('googleplaystore.csv')
android=list(reader(google_store))
android_header=android[0]
android=android[1:]


def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

print(ios_header)
print('\n')
explore_data(ios,0,5, True)
print('\n')
print(android_header)
print('\n')
explore_data(android,0,5, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']


Number of rows: 7197
Number of columns: 16


['App', 'Cate

Row 10472 miss category section, that's why we would exclude it from farther analysis 

In [2]:
del android[10472]

Google Play data set has not only missing errors, but double entries also. It could be basically showed at this example:

In [3]:
for app in android:
    if app[0]=='Facebook':
        print(app)

['Facebook', 'SOCIAL', '4.1', '78158306', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'August 3, 2018', 'Varies with device', 'Varies with device']
['Facebook', 'SOCIAL', '4.1', '78128208', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'August 3, 2018', 'Varies with device', 'Varies with device']


### Now let's count the amount of duplicates!


In [4]:
unique_apps=[]
duplicates=[]

for app in android:
    name=app[0]
    if name in unique_apps:
        duplicates.append(name)
    else:
        unique_apps.append(name)
        
print('Here is the number of duplicates:', len(duplicates))
print('\n')
print(duplicates[:10])

Here is the number of duplicates: 1181


['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack']


```
The main difference between duplicates appears to be the number of reviewers. Information that was gatherned later, would have more reviewes and that should be the basis for farther data cleaning
```
* In so doing we looped through the list and added in dictionary only the one with the highest value
* If there were no such a key we added it with a current value and updated it in future in case of encountering higher value

In [5]:
reviews_max={}
for app in android:
    name=app[0]
    n_reviews=float(app[3])
    if name in reviews_max and reviews_max[name]<n_reviews:
        reviews_max[name]=n_reviews
    if name not in reviews_max:
        reviews_max[name]=n_reviews
        
print(len(reviews_max))    

9659


At this step we just added row in the list if it matches the highest value from the dictionary and also compared it with the list of already added apps(rows)

In [6]:
android_clean=[]
already_added=[]

for app in android:
    name=app[0]
    n_reviews=float(app[3])
    if n_reviews==reviews_max[name] and name not in already_added:
        android_clean.append(app)
        already_added.append(name)
print(len(android_clean))

9659


The function below enables us to identify wether an app's name is in english or not (on the basis of checking all characters for belonging to english symbols in ASCII system)

In [7]:
def english_abc(string):
    
    for char in string:
        if ord(char)>127:
            return False
    return True

print(english_abc('Instagram'))
print(english_abc('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(english_abc('Docs To Go™ Free Office Suite'))
print(english_abc('Instachat 😜'))


True
False
False
False


Below you could slightly modified function that marks a word as a non enlish if there are more than 3 elements that exceed 127 value in ASCII system (which stands for basic english alphabet indexation)

In [8]:
def is_english(string):
    count=0
    for char in string:
        if ord(char)>127:
            count+=1
        if count>3:
            return False
        
    return True

print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))


True
False
True
True


Now let's create a new dataset that would consist from english names only.
* Loop through dataset and filter it with our is_english() function

In [9]:
ios_eng=[]
android_eng=[]

for app in android_clean:
    name=app[0]
    if is_english(name)==True:
        android_eng.append(app)
        
for app in ios:
    name=app[1]
    if is_english(name)==True:
        ios_eng.append(app)
        
    
explore_data(ios_eng,0,2,True)
explore_data(android_eng,0,2,True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


Number of rows: 6183
Number of columns: 16
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 9614
Number of columns: 13


As long as we are intersted in free apps only, we should filter our data again and exlude all paid applications. 

In [10]:
android_free=[]
ios_free=[]

for app in android_eng:
    price = app[7]
    if price == '0':
        android_free.append(app)
for app in ios_eng:
    price=app[4]
    if price=='0.0':
        ios_free.append(app)

print(len(android_free))
print(len(ios_free))

8864
3222


In [11]:
print(ios_header)
print('\n')
print(android_header)
# this function would identify index of column in header
def ident_column(column,header): 
    count=0
    for elem in header:
        count+=1
        if elem==column:
            return count-1
print('\n')
print(ident_column('prime_genre',ios_header),' is index of needed column')
print(ident_column('Genres',android_header),'is index of needed column')
print(ident_column('Category',android_header),'is index of needed column')

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


11  is index of needed column
9 is index of needed column
1 is index of needed column


Games are predominant section in english speaking section of an ios application storage. We could see that most apps are designed for entertaiment (2 predominant sections are Games & Enterteiment). The other section which could be compatible with enterteiment apps is bussines and tools apps, which fall into such popular categories as Photo&Video, Education and etc.
We couldn't say which future app would be most desirable to create on the basis of frequency table only. To make such conclusions we need to analyse quantity of users that these apps are downloaded and used by. 

In [12]:
def freq_table(dataset,index):
    fr_table={}
    total=0
    for row in dataset:
        total+=1
        fr_subj=row[index]
        if fr_subj in fr_table:
            fr_table[fr_subj]+=1
        else:
            fr_table[fr_subj]=1
    for elem in fr_table:
        fr_table[elem]=float((fr_table[elem]/total)*100)
    return fr_table
#print(freq_table(ios_free,11))
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])
#frequency table for prime_genre section (ios apps)
display_table(ios_free,11)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


In [13]:
#frequency table for category section (android_apps)
display_table(android_free,1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

In [14]:
#frequency table for category section (android_apps)
display_table(android_free,9)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

In [27]:
genre_fr=freq_table(ios_free,11)
for genre in genre_fr:
    total=0
    len_genre=0
    for app in ios_free:
        genre_app=app[11]
        if genre_app==genre:
            n_rating=float(app[5])
            total+=n_rating
            len_genre += 1
    avg_num_ratings = total/len_genre
    print(genre,' ',avg_num_ratings)

Games   22788.6696905016
Shopping   26919.690476190477
Utilities   18684.456790123455
Social Networking   71548.34905660378
Sports   23008.898550724636
Finance   31467.944444444445
Lifestyle   16485.764705882353
Business   7491.117647058823
Photo & Video   28441.54375
Book   39758.5
Education   7003.983050847458
Entertainment   14029.830708661417
Weather   52279.892857142855
Productivity   21028.410714285714
Travel   28243.8
Medical   612.0
Catalogs   4004.0
News   21248.023255813954
Navigation   86090.33333333333
Food & Drink   33333.92307692308
Reference   74942.11111111111
Music   57326.530303030304
Health & Fitness   23298.015384615384


In [37]:
category_fr=freq_table(android_free,1)
for category in category_fr:
    total=0
    len_category=0
    for app in android_free:
        genre_app=app[1]
        if genre_app==category:
            n_installs=app[5]
            n_installs.replace(',','')
            n_installs.replace('+','')
            n_installs=float(n_installs)
            total+=n_installs
            len_category+=1
    avg_num_install=total/len_category
    print(category,' ',avg_num_install)

ValueError: could not convert string to float: '1,000,000+'